<a href="https://colab.research.google.com/github/jpo85/meshroom_colab/blob/main/runtime_script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fotogrammetria tramite Google Cloud & Meshroom

Questa pagina web si prefigge lo scopo di facilitare l'elaborazione di un dataset di foto per tentare una prima elaborazione e ricostruzione fotogrammetrica. Gli strumenti che vengono utilizzati per eseguire l'elaborazione sono il servizio Google colab e Meshroom.

Il servizio Google Colab dove è anche ospitata la seguente pagina web, ha lo scopo di eseguire in cloud gli script shell e python per l'elaborazione delle immagini e la creazione della mesh 3d finale.
Google Colab è una piattaforma di google, nata principalmente per eseguire quelle operazioni di machine learning che richiedono prestazioni e potenza di calcolo elevata. Nel nostro caso specifico lo utilizzeremo per fini alternativi, e verranno messe a disposizione dall'infrastruttura cloud le seguenti potenze di calcolo:

*   32 GB di RAM
*   GPU con 16GB di RAM dedicata e funzionalità CUDA
*   Circa 40GB su filesystem

Meshroom è un software con licenza opensource che consente tramite foto e video di effettuare una ricostruzione fotogrammetrica, basandosi sul framework AliceVision. L'interfaccia di utilizzo è basata sulla logica dei workflow, in cui ogni blocco è collegato al successivo e salvando su una folder l'elaborazione temporanea ed infine generando una mesh poligonale. Questo runtime coonsente eventualmente anche di utilizzare un workflow custom in cui è possibile avere più risultati diversi.
Si può ad esemopio aggiungere un workflow di post produzione che effettui un Meshdecimate + Retexturing + Retopology della mesh, oppure esportare la pointcloud prima che venga creata la mesh.
Aggiungere anche un workflow che esporti la mesh finale ed effettui l'upload su sketchfab.

Per fare tutto ciò basta creare un workflow custom tramite gui dalla propria installazione meshroom su pc, eportarla in formato .mg, copiarla su gdrive ed inserire l'opzione nel comando dello script.





# Come funziona

la logica di questa guida è innanzitutto capire che non si tratta di una semplice pagina web statica ma di una vera e propria guida stepbystep dove dopo ogni spiegazione è presente direttamente il codice gà scritto da eseguire. Su ogni blocco è già scritto tutto il codice necessario per una elaborazione standard. Il workflow standard che viene eseguito in questa guida consente di prendere una dataset di foto e generare un file obj 3d con le sue relative texture.
il risutato finale sarà:

*   1 file in formato .obj
*   vari file di texture in formato .jpeg
*   1 file con la libreria dei materia in formato .mtl

Il codice presente è aperto perciò è modificabile a seconda delle proprie esigenze. Ad esempio se si dispone di un workflow meshroom custom si può inserire il comando per l'importazione del workflow specifico, e far eseguire quello anzichè quello di default del software.






# Prerequisiti

Lo script per essere eseguito prevede una minima configurazione iniziale e non necessita successivamente di scrivere codice.
Come prerequisito serve avere un'account google drive con spazio a sufficienza per fare l'upload delle foto e salvare la mesh finale. Lo script infatti utilizza come input e output due folder di google drive, le quali dovranno essere create a mano per consentire al codice di leggere le foto, elaborarle e salvare la mesh finale.

# Esecuzione StepByStep

Di seguito sono riportati una serie di passaggi da eseguire uno dopo l'altro che guideranno l'utente nell'elaborazione fotogrammetrica.

**0. Configura il runtime in modalità GPU e fai il check**

Questo blocco una volta eseguito consente di verificare se il runtime è configurato con l'utilizzo della GPU. Nel caso in cui l'esito fosse negativo, si riporta di seguito come abilitare tale funzione. Si ricorda che la GPU è fondamentale per una esecuzione più veloce dello script.

*abilitare le GPU per il notebook:
Vai Modifica → Impostazioni blocco note
Seleziona GPU dal menu a discesa Acceleratore hardware
Successivamente, confermeremo che possiamo connetterci alla GPU con tensorflow*



In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

**1. Connetti il tuo account di Google Drive**

come prima cosa devi aver creato nel filesystem di gdrive sulla root, due folder chiamate /input e /output


*   dentro /input devi fare l'upload del dataset di foto
*   dentro /output a fine ricostruzione troverai la mesh + textures + mtl

eseguendo il codice ti verrà richiesto l'authorization code per autorizzare il runtime ad accedere al filesystem di gdrive. Clicca sul link, copia la key generata e premi invio.

una volta eseguito verrà fatto un mountpoint del tuo gdrive dentro la folder /content/drive/MyDrive del runtime. 


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

**2. Crea le folder necessarie e fai il download di meshroom:**

Questo codice una volta eseguito crea due folder necessarie per ospitare l'eseguibile di meshroom zippato e scompattarlo.

*   /content/temp - contiene il pacchetto meshroom scaricato
*   /content/meshroom - contiene tutti gli eseguibili di meshroom scompattati e 
pronti per essere eseguiti

la versione utilizzata di meshroom è la v2019.2.0 for linux





In [ ]:
%cd /content
!mkdir temp
!mkdir meshroom
%cd temp
!wget -N https://github.com/alicevision/meshroom/releases/download/v2019.2.0/Meshroom-2019.2.0-linux.tar.gz
!tar -xvf Meshroom-2019.2.0-linux.tar.gz -C ../meshroom

**3. Esegui Meshroom**

Se i precendenti blocchi sono stati eseguiti senza errori, si può avvire il blocco che effettua la vera e propria ricostruzione fotogrammetrica. Il workflow di default salverà ogni output parziale dentro la folder:

* /tmp/MeshroomCache

l'output finale, come riportato precedentemente, sarà salvato dentro:

* /content/drive/MyDrive/output


**NOTA BENE: NON CHIUDERE LA PAGINA WEB SE SI VUOLE RIMANERE CONNESSI PER VEDERE L'AVANZAMENTO. ANCHE CHIUDENDO LA PAGINA IL RUNTIME CONTINUERA L'ESECUZIONE PER LE SUCCESSIVE 12 ORE. PASSATE 12 ORE VERRA' CANCELLATO TUTTO IL FILESYSTEM DEL RUNTIME E VERRA' STOPPATA L'ESECUZIONE DELLO SCRIPT NEL CASO IN CUI NON SIA ANCORA TERMINATO**



In [ ]:
!/content/meshroom/Meshroom-2019.2.0/meshroom_photogrammetry --input '/content/drive/MyDrive/input' --output '/content/drive/MyDrive/output'

**4. Preview Mesh usando Trimesh** 

ancora in fase sperimentale

In [ ]:
!pip install numpy
!pip install trimesh
%cd /content/drive/output

lancia preview

In [ ]:
import numpy as np
import trimesh
mesh = trimesh.load_mesh('texturedMesh.obj')
mesh.show()

*version 1.4*

*Author: jacopo pica*

*jpo85@hotmail.it*

*https://it.linkedin.com/in/jacopo-pica-8164a4161*

*https://github.com/jpo85*
